In [1]:
print('hello world')

hello world


## Qwen/Qwen3-Coder-480B-A35B-Instruct + GPT-4o

- 选表：GPT-4o
- sql生成：Qwen/Qwen3-Coder-480B-A35B-Instruct
- 双模型协作

In [54]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, PromptTemplate
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
import pandas as pd
import os

# === 示例导入 ===

from examples.concert_singer_examples import concert_singer_examples
from examples.customers_and_products_contacts_examples import customers_and_products_contacts_examples
from examples.driving_school_examples import driving_school_examples
from examples.formula_1_examples import formula_1_examples
from examples.hospital_1_examples import hospital_1_examples
from examples.riding_club_examples import riding_club_examples
from examples.soccer_1_examples import soccer_1_examples
from examples.wine_1_examples import wine_1_examples
from examples.world_1_examples import world_1_examples
from examples.bike_1_examples import bike_1_examples


In [55]:
# === 路径和模型设置 ===
script_dir = os.getcwd()
db_paths = {
    "bike_1": os.path.join(script_dir, "database", "bike_1", "bike_1.sqlite"),
    "concert_singer": os.path.join(script_dir, "database", "concert_singer", "concert_singer.sqlite"),
    "customers_and_products_contacts": os.path.join(script_dir, "database", "customers_and_products_contacts", "customers_and_products_contacts.sqlite"),
    "driving_school": os.path.join(script_dir, "database", "driving_school", "driving_school.sqlite"),
    "formula_1": os.path.join(script_dir, "database", "formula_1", "formula_1.sqlite"),
    "hospital_1": os.path.join(script_dir, "database", "hospital_1", "hospital_1.sqlite"),
    "riding_club": os.path.join(script_dir, "database", "riding_club", "riding_club.sqlite"),
    "soccer_1": os.path.join(script_dir, "database", "soccer_1", "soccer_1.sqlite"),
    "wine_1": os.path.join(script_dir, "database", "wine_1", "wine_1.sqlite"),
    "world_1": os.path.join(script_dir, "database", "world_1", "world_1.sqlite")
}

In [56]:
llm = ChatOpenAI(
    model='gpt-4o',
    base_url="https://api.gpt.ge/v1",
    api_key="sk-zZLix18RO29mDPcI7dB4BdB8392241D4B56e76F0CbA3Cf76",
    temperature=0.3
)


In [57]:
from openai import OpenAI
client = OpenAI(
    base_url="https://api-inference.modelscope.cn/v1/",
    api_key="e32ccce5-592d-41b5-81f6-782c3dd0210c",  
)

In [58]:
# === 表结构描述加载 ===
def get_table_details(db_id: str):
    csv_path = os.path.join(script_dir, f"{db_id}_table_description.csv")
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"❌ 找不到表结构描述文件：{csv_path}")
    df = pd.read_csv(csv_path)
    return "\n".join([f"表名：{row['Table']}\n表的基本概述：{row['Description']}\n" for _, row in df.iterrows()])

In [59]:
# ✅ 新版选表 Pydantic 模型
class TableList(BaseModel):
    tables: List[str] = Field(description="List of relevant table names")

In [60]:
def select_tables_via_parser(llm, question: str, db_id: str, db: SQLDatabase) -> List[str]:
    table_description_text = get_table_details(db_id)
    all_tables = db.get_usable_table_names()

    parser = PydanticOutputParser(pydantic_object=TableList)

    system_table_msg = f"""你是一个严谨的数据库表选择专家。以下是数据库中所有表的结构描述：

{table_description_text}

请根据用户问题，从这些表中选择可能相关的表名，并返回JSON格式，务必遵守以下要求：

1. 必须至少选择一个表，即使不确定是否使用，也应包含可能相关的表。
2. 只返回你认为可能相关的表名列表，不包含额外解释。
3. 表名必须来自于：{', '.join(all_tables)}。
4. 输出格式：{{"tables": ["表名1", "表名2"]}}
"""

    prompt = PromptTemplate(
        template="{system_table_msg}\n用户问题：{question}\n\n{format_instructions}",
        input_variables=["system_table_msg", "question"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    formatted_prompt = prompt.format_prompt(system_table_msg=system_table_msg, question=question)

    try:
        output = llm.invoke(formatted_prompt.to_string())
        result = parser.parse(output.content)
        selected = [t for t in result.tables if t in all_tables]
        print(f"\n✅ 模型选择的相关表：{selected}")
        return selected
    except Exception as e:
        print("⚠️ 表选择失败，使用 fallback：全部表")
        return all_tables
    

In [ ]:
# === few-shot ===
def build_few_shot_prompt(db_id: str):
    example_map = {
        "bike_1": bike_1_examples,
        "concert_singer": concert_singer_examples,
        "customers_and_products_contacts": customers_and_products_contacts_examples,
        "driving_school": driving_school_examples,
        "formula_1": formula_1_examples,
        "hospital_1": hospital_1_examples,
        "riding_club": riding_club_examples,
        "soccer_1": soccer_1_examples,
        "wine_1": wine_1_examples,
        "world_1": world_1_examples
    }

    selected_examples = example_map.get(db_id)
    if not selected_examples:
        raise ValueError(f"未知数据库 ID：{db_id}")

    vectorstore = Chroma()
    vectorstore.delete_collection()

    example_selector = SemanticSimilarityExampleSelector.from_examples(
        selected_examples,
        OpenAIEmbeddings(
            base_url="https://api.gpt.ge/v1",
            api_key="sk-zZLix18RO29mDPcI7dB4BdB8392241D4B56e76F0CbA3Cf76",
            model="text-embedding-3-small",
        ),
        vectorstore,
        k=2,
        input_keys=["input"]
    )

    example_prompt = ChatPromptTemplate.from_messages([
        ("human", "问题：{input}\n推理方式：{reasoning_type}\n常识知识：{commonsense_knowledge}\n对应生成的 SQL 查询语句："),
        ("ai", "{query}")
    ])

    return FewShotChatMessagePromptTemplate(
        example_selector=example_selector,
        example_prompt=example_prompt,
        input_variables=["input", "reasoning_type", "commonsense_knowledge"]
    )

def generate_few_shot_text(few_shot_prompt, question: str):
    selected_examples = few_shot_prompt.example_selector.select_examples({"input": question})
    texts = []
    for ex in selected_examples:
        texts.append(
            f"Human: 问题：{ex['input']}\n推理方式：{ex['reasoning_type']}\n常识知识：{ex['commonsense_knowledge']}\n对应生成的 SQL 查询语句：\nAI: {ex['query']}"
        )
    return "\n\n".join(texts)

In [ ]:
from typing import Tuple
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "M-Schema"))
from schema_engine import SchemaEngine
from typing import List, Tuple

from sqlalchemy import create_engine
import os
def extract_mschema_str(db_id: str, selected_tables: List[str]) -> Tuple[str, str]:
    db_path = db_paths[db_id]
    abs_path = os.path.abspath(db_path)
    db_engine = create_engine(f"sqlite:///{abs_path}")

    print(f"\n🔧 [调试] 连接数据库：{abs_path}")
    print(f"🔧 [调试] 模型选出的相关表：{selected_tables}")

    schema_engine = SchemaEngine(engine=db_engine, db_name=db_id)
    mschema = schema_engine.mschema

    print(f"🔧 [调试] m-schema 原始表数量：{len(mschema.tables)}")
    print("🔧 [调试] 原始表名列表：", list(mschema.tables.keys()))

    def table_name_without_prefix(fullname: str) -> str:
        # 移除可能的 "main." 或类似前缀
        return fullname.split('.')[-1]

    # 只保留选中的表（忽略前缀）
    mschema.tables = {
        name: table for name, table in mschema.tables.items()
        if table_name_without_prefix(name) in selected_tables
    }

    print(f"✅ [调试] 筛选后表数量：{len(mschema.tables)}")
    print("✅ [调试] 保留表名列表：", list(mschema.tables.keys()))

    mschema_str = mschema.to_mschema()

    return mschema_str, db_engine.dialect.name


In [63]:
nl2sqlite_template_cn = """
你是一名资深 {dialect} 数据库专家，任务是：根据给定的【数据库结构】和【用户问题】，生成**符合 {dialect} 语法且能直接执行的 SQL 查询语句**，用以准确回答用户问题。

请严格按照如下步骤思考和生成 SQL：

1. 阅读并理解【数据库schema】中各表及字段的含义；
2. 阅读【参考信息】，其中包含与问题相似的示例，提供了可能的推理类型和常识知识；
3. 根据【用户问题】中的关键词，分析应使用的字段；
4. **构造多层嵌套查询时，请确保每层查询都包含外层可能使用的所有字段，避免出现找不到字段的情况**。
5. **要分析清楚用户的问题一共有几个问题，确保问题包含的字段不要多也不能少，都能在 SQL 中找到**。
6. **最终仅输出一条 SQL 查询语句，禁止生成任何注释、解释或额外内容。**

推理类型说明（用于参考信息中示例问题）：
- + ：需要执行加法运算
- - ：需要执行减法运算
- * ：需要执行乘法运算
- / ：需要执行除法运算
- C ：需要应用常识推理（可参考示例中的 commonsense 内容）
- H ：需要基于假设或特定条件推理

【数据库schema】
{db_schema}

【参考信息】
{evidence}

【用户问题】
{question}

```sql
"""

In [64]:
def generate_sql_only(db_id: str, question: str) -> str:
    if db_id not in db_paths:
        raise ValueError(f"未知数据库 ID：{db_id}")

    # 1. 表选择
    db = SQLDatabase.from_uri(f"sqlite:///{db_paths[db_id]}")
    selected_tables = select_tables_via_parser(llm, question, db_id, db)
    print(f"✅ [表选择] 模型选出的相关表: {selected_tables}")

    # 2. Few-shot 示例（参考信息）
    few_shot_prompt = build_few_shot_prompt(db_id)
    evidence = generate_few_shot_text(few_shot_prompt, question)

    # 3. m-schema 构建
    mschema_str, dialect = extract_mschema_str(db_id, selected_tables)

    # 4. Prompt 构造
    prompt = nl2sqlite_template_cn.format(
        dialect=dialect,
        db_schema=mschema_str,
        question=question,
        evidence=evidence
    )
    print("📄 Prompt 内容 ↓↓↓\n", prompt)

    # 5. OpenAI API 调用（使用 ModelScope）
    messages = [
        {"role": "system", "content": "你是一个精通 SQLite 的专家,根据给定的【数据库结构】和【用户问题】，生成**符合 SQLite 语法且能直接执行的 SQL 查询语句**"},
        {"role": "user", "content": prompt}
    ]

    response = client.chat.completions.create(
        model='Qwen/Qwen3-Coder-480B-A35B-Instruct',
        messages=messages,
    )

    result = response.choices[0].message.content.strip()

    # 6. 清洗结果
    if result.startswith("```sql"):
        lines = result.splitlines()
        result = "\n".join(line for line in lines if not line.strip().startswith("```"))

    print("🧠 生成 SQL ↓↓↓\n", result)
    return result.strip()

In [65]:
def main():
    print("📦 交互式 NL2SQL 模式已启动（使用 QWEN-coder）")
    print("可用数据库：", ", ".join(db_paths.keys()))
    print("输入 q 退出。\n")
    while True:
        try:
            user_input = input("🔍 请输入：").strip()
            if user_input.lower() in ["q", "quit", "exit"]:
                print("👋 再见！")
                break
            if ":" not in user_input:
                print("⚠️ 格式错误，应为 '<db_id>: <问题>'，例如：bike_1: 每个终点站的最长持续时间")
                continue
            db_id, question = user_input.split(":", 1)
            sql = generate_sql_only(db_id.strip(), question.strip())
        except Exception as e:
            print("❌ 错误：", e)



In [66]:
if __name__ == "__main__":
    main()

📦 交互式 NL2SQL 模式已启动（使用 QWEN-coder）
可用数据库： bike_1, concert_singer, customers_and_products_contacts, driving_school, formula_1, hospital_1, riding_club, soccer_1, wine_1, world_1
输入 q 退出。



Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



✅ 模型选择的相关表：['station']
✅ [表选择] 模型选出的相关表: ['station']


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



🔧 [调试] 连接数据库：d:\PythonProject\livesqlbench\livesqlbench-main\database\bike_1\bike_1.sqlite
🔧 [调试] 模型选出的相关表：['station']
🔧 [调试] m-schema 原始表数量：4
🔧 [调试] 原始表名列表： ['main.station', 'main.status', 'main.trip', 'main.weather']
✅ [调试] 筛选后表数量：1
✅ [调试] 保留表名列表： ['main.station']
📄 Prompt 内容 ↓↓↓
 
你是一名资深 sqlite 数据库专家，任务是：根据给定的【数据库结构】和【用户问题】，生成**符合 sqlite 语法且能直接执行的 SQL 查询语句**，用以准确回答用户问题。

请严格按照如下步骤思考和生成 SQL：

1. 阅读并理解【数据库schema】中各表及字段的含义；
2. 阅读【参考信息】，其中包含与问题相似的示例，提供了可能的推理类型和常识知识；
3. 根据【用户问题】中的关键词，分析应使用的字段；
4. **构造多层嵌套查询时，请确保每层查询都包含外层可能使用的所有字段，避免出现找不到字段的情况**。
5. **要分析清楚用户的问题一共有几个问题，确保问题包含的字段不要多也不能少，都能在 SQL 中找到**。
6. **最终仅输出一条 SQL 查询语句，禁止生成任何注释、解释或额外内容。**

推理类型说明（用于参考信息中示例问题）：
- + ：需要执行加法运算
- - ：需要执行减法运算
- * ：需要执行乘法运算
- / ：需要执行除法运算
- C ：需要应用常识推理（可参考示例中的 commonsense 内容）
- H ：需要基于假设或特定条件推理

【数据库schema】
【DB_ID】 bike_1
【Schema】
# Table: main.station
[
(id:INTEGER, Primary Key, Examples: [2, 3, 4]),
(name:TEXT, Examples: [San Jose Diridon Caltrain Station]),
(lat:NUMERIC, Examples: [37.329732, 37.330698, 3